# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

In [2]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In [4]:
tf.__version__

'2.7.0'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [5]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [17]:
batch_size = 15
img_idx = [x for x in range(0,30,2)]
y = 160
z = 160
input_shape=(len(img_idx),y,z,3)

In [18]:
def one_batch_data_generator(source_path, t, batch, batch_size, img_idx, y, z, augment):
  batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
  batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output

  if augment:
    batch_data_augment = np.zeros((batch_size,len(img_idx),y,z,3))

  for folder in range(batch_size): # iterate over the batch_size
    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
        temp = cv2.resize(image,(z,y))
        temp = temp/255 #Normalize data

        batch_data[folder,idx,:,:,0] = (temp[:,:,0])  #feed in the image
        batch_data[folder,idx,:,:,1] = (temp[:,:,1])  #feed in the image
        batch_data[folder,idx,:,:,2] = (temp[:,:,2])  #feed in the image

        if augment:
          shifted = cv2.warpAffine(image, np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30,30)]]), (image.shape[1], image.shape[0]))

          gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

          x0, y0 = np.argwhere(gray > 0).min(axis=0)
          x1, y1 = np.argwhere(gray > 0).max(axis=0)

          # cropping the images to have the targeted gestures and remove the noise from the images.
          cropped=shifted[x0:x1,y0:y1,:]

          temp=cv2.resize(cropped,(z,y))
          temp = temp/255 #Normalize data

          batch_data_augment[folder,idx,:,:,0] = (temp[:,:,0])
          batch_data_augment[folder,idx,:,:,1] = (temp[:,:,1])
          batch_data_augment[folder,idx,:,:,2] = (temp[:,:,2])

    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

  if augment:
    batch_data = np.concatenate([batch_data,batch_data_augment])
    batch_labels = np.concatenate([batch_labels,batch_labels])

  return (batch_data, batch_labels)

In [19]:
def generator(source_path, folder_list, batch_size, augment=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
          batch_data, batch_labels = one_batch_data_generator(source_path, t, batch, batch_size, img_idx, y, z, augment)
          yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        last_batch_size = len(folder_list)%(batch_size)
        if last_batch_size != 0: # we check if any folders remaining after the last batch
          batch_data, batch_labels = one_batch_data_generator(source_path, t, num_batches, last_batch_size, img_idx, y, z, augment)
          yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GRU
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers

In [22]:
model = Sequential()
model.add(Conv3D(16, 3, padding='same',
          input_shape=(len(img_idx),y,z,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, 2, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, 2, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, 2, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2),padding='same'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(5,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [23]:
optimiser = optimizers.Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 15, 160, 160, 16)  1312      
                                                                 
 activation (Activation)     (None, 15, 160, 160, 16)  0         
                                                                 
 batch_normalization_2 (Batc  (None, 15, 160, 160, 16)  64       
 hNormalization)                                                 
                                                                 
 max_pooling3d (MaxPooling3D  (None, 7, 80, 80, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 7, 80, 80, 32)     4128      
                                                                 
 activation_1 (Activation)   (None, 7, 80, 80, 32)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [24]:
train_generator = generator(train_path, train_doc, batch_size, augment = True)
val_generator = generator(val_path, val_doc, batch_size)

In [25]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
earlystop = EarlyStopping( monitor="val_loss", min_delta=0,patience=10,verbose=1)
callbacks_list = [checkpoint, LR, earlystop]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [26]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [27]:
import time

start = time.time()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0,max_queue_size=3000)

end = time.time()
print("Time taken to train module: ", (end - start), " seconds")

/tmp/ipykernel_303/1846909349.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 15
Epoch 1/25
45/45 [==============================] - ETA: 0s - loss: 1.5715 - categorical_accuracy: 0.4072Source path =  /home/datasets/Project_data/val ; batch size = 15

Epoch 00001: saving model to model_init_2023-08-3106_19_29.364795/model-00001-1.57154-0.40724-3.11885-0.21000.h5
45/45 [==============================] - 87s 2s/step - loss: 1.5715 - categorical_accuracy: 0.4072 - val_loss: 3.1189 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/25
45/45 [==============================] - ETA: 0s - loss: 1.1091 - categorical_accuracy: 0.5792
Epoch 00002: saving model to model_init_2023-08-3106_19_29.364795/model-00002-1.10907-0.57919-4.44324-0.21000.h5
45/45 [==============================] - 84s 2s/step - loss: 1.1091 - categorical_accuracy: 0.5792 - val_loss: 4.4432 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/25
45/45 [==============================] - ETA: 0s - loss: 0.8913 - categorical_accuracy: